In [1]:
# wir benötigen wieder ein paar python imports...
import sys
from os import environ
import os
import uuid
sys.path.append(os.path.abspath(".."))

from sparql_queries import *
from connector_client import ConnectorClient
from connector.messages.datamodel_utils import SubscriptionRegisterRequest, SubscriptionUnregisterRequest # generic messages within the connector_base repo

client = ConnectorClient(bootstrap_servers=[environ.get("KAFKA_BROKER")])

In [2]:
# graphdb queries, um den richtigen connector zu identifizieren
await client.load_connector_config()
await client.return_connectors()

dict_keys(['milling_station', 'injection_molding', 'assembly_station'])

In [3]:
# wechsel an den ausgewählten, richtigen connector
await client.switch_connector('milling_station')

In [6]:
# definieren einer Subscribe-Nachricht für einen OPC-UA-Server, welche bereits als pydantic model oder als json schema gegeben ist
subscription_id = uuid.uuid4()
print(f"Subscription Identifier: {subscription_id}")
example_subscribe_payload = SubscriptionRegisterRequest(
    datapoint_identifier=["ns=2;i=62"],
    device_origin="client",
    subscription_identifier=subscription_id
)

Subscription Identifier: b545fd3f-833d-43d1-931c-8e78566b580f


In [7]:
# senden der Nachricht und darstellen der Rückmeldung
output = await client.subscribe_data_source(base_payload=example_subscribe_payload)

Correlation_id of sent message: e4500be7-2fd6-4db6-a94b-87ab80d97a37
waiting for a message
Received message
{'correlation_id': 'e4500be7-2fd6-4db6-a94b-87ab80d97a37',
 'message_id': 'cf7bba46-0454-45eb-b3b4-36acbff20391',
 'payload': {'base_payload': {'device_origin': 'connector_opcua',
                              'response': 'Worked',
                              'type': 'SubscriptionRegisterResponse'},
             'type': 'CommandResponse'},
 'service_id': 'milling_station',
 'status_code': 'Valid',
 'timestamp': '2025-04-28T20:06:55.066094',
 'type': 'ResponseMessage',
 'version': '1.0.0'}
message type: <class 'connector.messages.datamodel_utils.SubscriptionRegisterResponse'>
Matching response received:
{
    "type": "SubscriptionRegisterResponse",
    "device_origin": "connector_opcua",
    "response": "Worked"
}


In [21]:
SubscriptionRegisterRequest.model_json_schema()

{'properties': {'type': {'const': 'SubscriptionRegisterRequest', 'default': 'SubscriptionRegisterRequest', 'title': 'Type', 'type': 'string'}, 'device_origin': {'description': 'The origin of the device that the payload is sent from independent of the path in between', 'title': 'Device Origin', 'type': 'string'}, 'datapoint_identifier': {'description': 'Identifier for a DataPoint of the device', 'items': {'type': 'string'}, 'title': 'Datapoint Identifier', 'type': 'array'}, 'subscription_identifier': {'description': 'Identifier for the subscription', 'format': 'uuid', 'title': 'Subscription Identifier', 'type': 'string'}}, 'required': ['device_origin', 'datapoint_identifier', 'subscription_identifier'], 'title': 'SubscriptionRegisterRequest', 'type': 'object'}